# Tutorial: converting, writing, and reading with heparchy

Here's a quick (very incomplete) primer on using `heparchy`'s utilities to convert, write, and read files in a hierarchical and high performance way.

(Proper Sphinx documentation coming soon.)

In [1]:
from tqdm import tqdm # some nice progress bars :)

## Write events hierarchically under processes within a database file

In [2]:
from heparchy.write import HepWriter

As long as you provide `HepWriter` with `numpy` arrays in the correct shape and data type, you can source your data however you want. In this case, we make use of the built-in HepMC file parser, `heparchy.hepmc.HepMC`.

In [3]:
from heparchy.hepmc import HepMC

The `heparchy.hepmc.HepMC` file parser returns an object whose `data` property is a `heparchy.data.ShowerData` object. This has some convenience methods which traverse the shower looking for a user defined signal vertex, and then follows one of the produced particles, identifying all of its descendants with a boolean mask. To make use of this functionality during the data conversion, we will also import `heparchy.data.SignalVertex`, and define some vertices for this process.

In [4]:
from heparchy.data import SignalVertex

In [5]:
signal_vertices = [
    SignalVertex( # top decay
        incoming=[6], outgoing=[24,5], # defines the vertex
        follow=[24,5] # specifies which of the outgoing particles to track in the shower
    ), 
    SignalVertex( # anti-top decay
        incoming=[-6], outgoing=[-24,-5],
        follow=[-5] # we can be selective about which outgoing particles to follow
    ),
]

Heparchy uses context managers and iterators to improve safety, speed, and remove boilerplate. This does lead to a lot of nesting, but the result is fairly intuitive.

1. create a file to store the data
2. add "processes" to that file (however you want to define them, _eg._ `p p > t t~`)
3. within those processes nest events, each of which contain datasets

There are context managers for each of those stages which handle the fiddly bits and standardise the process. The returned objects then provide methods to write the datasets, as in the example below.

The example below also contains the `HepMC` file parser, which itself opens HepMC files by use of a context manager, and the returned object may be iterated over all of the events. So that's another two layers of nesting (yay), but pretty convenient.

In [7]:
with HepWriter('showers.hdf5') as hep_file: # first we create the file
    with hep_file.write_process('top') as process: # then write a process
        with HepMC('/home/jlc1n20/messy/test.hepmc') as raw_file: # load in data to convert from HepMC
            for shower in tqdm(raw_file): # iterate through the events in the HepMC file
                signal_masks = shower.signal_mask(signal_vertices)
                # signal_masks is a list in same order as signal_vertices
                # each element is a dictionary, keyed by pdg code of followed particle
                W_mask = signal_masks[0][24]
                b_mask = signal_masks[0][5]
                anti_b_mask = signal_masks[1][-5]
                with process.new_event() as event: # create event for writing
                    # add datasets - each is optional!
                    event.edges(shower.edges) # can omit if only storing final state
                    event.pmu(shower.pmu)
                    event.pdg(shower.pdg)
                    event.mask(name='final', data=shower.final)
                    event.mask(name='W_mask', data=W_mask)
                    event.mask(name='b_mask', data=b_mask)
                    event.mask(name='anti_b_mask', data=anti_b_mask)

4999it [09:42,  8.58it/s]


## Read data from heparchy format

In [8]:
from heparchy.read import HepReader

### Iteratively read all events of a given process

Reading data follows a similar hierarchical structure to writing data, as above.

1. open the heparchy data file
2. read processes given by name
3. iterate over the nested events, extracting their datasets

The first two of these tasks are handled with context managers, but the final task is achieved simply by iterating over the process object, which provides event objects with properties and methods that efficiently read from the heparchy file.

In [9]:
with HepReader('showers.hdf5') as hep_file:
    with hep_file.read_process(name='top') as process:
        for shower in tqdm(process):
            pmu = shower.pmu
            pdg = shower.pdg
            num_pcls = shower.count
            name = shower.name
            edges = shower.edges
            final = shower.mask('final')
            W_mask = shower.mask('W_mask')

100%|█████████████████| 4999/4999 [00:09<00:00, 538.91it/s]


12 seconds - not a bad speedup having needed 11 minutes to read the data from HepMC!

### Read individual events

If you need only to access one event at a time, or are using this library within dataloaders which extract datasets in their own order _eg._ for `pytorch`, you need not iterate over the process, and instead can use the `read_event` method.

In [10]:
with HepReader('showers.hdf5') as hep_file:
    with hep_file.read_process(name='top') as process:
        num_events = len(process)
        shower = process.read_event(129)
        pmu = shower.pmu
        pdg = shower.pdg
        num_pcls = shower.count
        name = shower.name
        edges = shower.edges
        final = shower.mask('final')
        W_mask = shower.mask('W_mask')
        b_mask = shower.mask('b_mask')
        anti_b_mask = shower.mask('anti_b_mask')

### Sanity check: the extracted data

Just to calm any misgivings about what is contained in all of these properties, it's all just strings, integers, and numpy arrays. See below.

In [11]:
name

'event_000000129'

In [12]:
num_pcls

1465

In [13]:
pmu

array([(0.        ,  0.        ,  6.49999993e+03, 6.50000000e+03),
       (0.80054601,  4.50754949,  1.54891257e+02, 1.54958899e+02),
       (1.50883326, -0.66960864,  1.99121135e+01, 1.99804209e+01), ...,
       (0.53140697,  0.79801622, -1.50074032e+00, 1.78085506e+00),
       (0.04391901,  0.00948937, -2.50044546e-02, 5.14212982e-02),
       (0.37794272,  0.56994011, -5.10351653e-01, 8.53306066e-01)],
      dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('e', '<f8')])

In [14]:
pdg

array([2212,   21,   21, ...,   22,   22,   22], dtype=int32)

In [15]:
edges

array([(   0,   -1), (  -1,   -3), (  -1,   -4), ..., (-853, 1463),
       (-854, 1464), (-854, 1465)], dtype=[('in', '<i4'), ('out', '<i4')])

In [16]:
final

array([False, False, False, ...,  True,  True,  True])

In [17]:
W_mask

array([False, False, False, ..., False, False, False])

## What next?

You can, of course, now do whatever you want with this data. Below I list some useful idioms for handling the data afterwards.

### Combining masks

In [18]:
import numpy as np

Now that we have access to some boolean masks over the data, we can combine them to perform simple queries over the particle data.

_eg._ to get the final state particles which descended from the W, simply perform a bitwise `and`.

In [19]:
W_final = np.bitwise_and(W_mask, final)
pmu[W_final] # extract momenta of final W descendants

array([(-4.24904107e-01,  9.07143346e-01, -3.55338172e+01, 3.55479341e+01),
       (-2.22183642e-05,  4.62683179e-05, -1.85672311e-03, 1.85743240e-03),
       (-8.65513477e+01, -1.77366849e+01,  2.11801548e+00, 8.83754025e+01)],
      dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('e', '<f8')])

If you want to do the same thing for the b-quark descendants, but also to remove neutrinos because they aren't going to show up in detector data, you can do a boolean comparison over the `pdg` output array, and perform the `and` operation over all three masks using the `ufunc.reduce` method. _ie._

In [20]:
neutrino_pdgs = [12, 14, 16]
neutrino_filter = ~np.isin(np.abs(pdg), neutrino_pdgs)
b_detectable = np.bitwise_and.reduce([b_mask, final, neutrino_filter])
pmu[b_detectable] # extract momenta of detectable b descendants

array([( 0.11847421, -3.82431983e-01, -1.06584314e+00, 1.14707966e+00),
       ( 0.93550098, -4.30468209e+00, -9.56356825e+00, 1.05302786e+01),
       ( 0.63358372, -4.00456359e+00, -8.13469408e+00, 9.09014221e+00),
       ( 0.48140494, -5.70424469e+00, -1.06098323e+01, 1.20657540e+01),
       ( 0.22495109, -1.02172372e+00, -1.77722080e+00, 2.06700650e+00),
       ( 0.01291836, -1.38563470e+00, -3.76169541e+00, 4.03908686e+00),
       ( 0.00601146,  3.04669205e-01, -1.11571458e+00, 1.16497136e+00),
       ( 0.55975262,  4.15843901e-01, -4.24927614e+00, 4.30837286e+00),
       ( 0.56481545,  1.17487632e+00, -4.66866625e+00, 4.84925512e+00),
       (-0.08362074,  2.64701094e-01, -1.49366548e+00, 1.52563935e+00),
       ( 0.74364107,  3.01304099e-01, -6.35378141e+00, 6.47260959e+00),
       ( 0.53905276, -1.14836693e+00, -6.00786156e+00, 6.14192191e+00),
       ( 0.23541338, -3.40396300e-01, -1.78209892e+01, 1.78505388e+01),
       ( 2.1792394 ,  1.90419880e-01, -3.23466530e+01, 3.2434150

### Querying events via DataFrames

While this is fine for basic manipulations, it does get rather messy when more detailed data extraction is required. As a convenience, `ShowerData` objects have a method which returns `pandas.DataFrame` object, which has extremely powerful vectorised aggregation and query methods.

In [21]:
from heparchy.data import ShowerData

In [22]:
with HepReader('showers.hdf5') as hep_file:
    with hep_file.read_process(name='top') as process:
        event = process.read_event(1202) # event number chosen at whim
        shower = ShowerData(
            edges=event.edges,
            pmu=event.pmu,
            pdg=event.pdg,
            final=event.mask('final'),
        )

shower_df = shower.to_pandas(data=['pdg', 'pmu', 'final', 'pt', 'eta', 'phi'])
shower_df

,pt,pdg,eta,final,phi,x,y,z,e
0,0.000000,2212,1.797693e+308,False,0.000000,0.000000,0.000000,6499.999932,6500.000000
1,2.547569,2,6.277804e+00,False,-2.357716,-1.804142,-1.798661,678.438124,678.442907
2,0.847329,21,2.105576e+00,False,0.584225,0.706791,0.467347,3.427470,3.530654
3,2.568915,21,4.873345e+00,False,1.077510,1.216441,2.262652,167.942687,167.962334
4,0.461121,21,7.094888e+00,False,2.792266,-0.433270,0.157825,278.006368,278.006751
...,...,...,...,...,...,...,...,...,...
2101,0.698772,22,-7.201081e-01,True,2.298916,-0.465010,0.521583,-0.547822,0.887914
2102,0.193734,22,-4.224109e-01,True,2.140502,-0.104497,0.163135,-0.084291,0.211276
2103,0.327677,22,-6.567684e-01,True,1.654239,-0.027311,0.326537,-0.231016,0.400925
2104,0.508834,22,-1.168402e+00,True,1.478327,0.046985,0.506660,-0.739331,0.897509


This reconstructs the same dataclass object that was being handed to us from the `heparchy.hepmc.HepMC` parser, hence we can compute signal masks for the event any time we like, not just when parsing HepMC files\*. Let's compute them again, this time following the W- as well, because now we've demonstrated the ability to be selective, it's just annoying not to have it. We can then add this to the DataFrame and do some glorious compound queries on the whole dataset!

---

\* The data doesn't need to be from a HepMC file at all. As long as you can format the shower into numpy arrays, you can pass it to the `ShowerData` object constructor.

In [23]:
signal_vertices = [
    SignalVertex(incoming=[6], outgoing=[24,5], follow=[24,5]), 
    SignalVertex(incoming=[-6], outgoing=[-24,-5], follow=[-24, -5]),
]

signal_masks = shower.signal_mask(signal_vertices)
shower_df['W'] = signal_masks[0][24]
shower_df['b'] = signal_masks[0][5]
shower_df['anti_W'] = signal_masks[1][-24]
shower_df['anti_b'] = signal_masks[1][-5]

shower_df

,pt,pdg,eta,final,phi,x,y,z,e,W,b,anti_W,anti_b
0,0.000000,2212,1.797693e+308,False,0.000000,0.000000,0.000000,6499.999932,6500.000000,False,False,False,False
1,2.547569,2,6.277804e+00,False,-2.357716,-1.804142,-1.798661,678.438124,678.442907,False,False,False,False
2,0.847329,21,2.105576e+00,False,0.584225,0.706791,0.467347,3.427470,3.530654,False,False,False,False
3,2.568915,21,4.873345e+00,False,1.077510,1.216441,2.262652,167.942687,167.962334,False,False,False,False
4,0.461121,21,7.094888e+00,False,2.792266,-0.433270,0.157825,278.006368,278.006751,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,0.698772,22,-7.201081e-01,True,2.298916,-0.465010,0.521583,-0.547822,0.887914,True,False,False,False
2102,0.193734,22,-4.224109e-01,True,2.140502,-0.104497,0.163135,-0.084291,0.211276,True,False,False,False
2103,0.327677,22,-6.567684e-01,True,1.654239,-0.027311,0.326537,-0.231016,0.400925,True,False,False,False
2104,0.508834,22,-1.168402e+00,True,1.478327,0.046985,0.506660,-0.739331,0.897509,True,False,False,False


Nice, eh?

If we wish to perform cuts on the data, for instance to filter out particles which wouldn't be observed in the final state, it is trivial to extract this data using `query`.

In [24]:
nu_pdgs = (12, 14, 16)
detect_df = shower_df.query('final and pt > 0.5 and abs(eta) < 2.5 and @nu_pdgs not in abs(pdg)')

In one line, we have extracted particles in the final state, while filtering out low transverse momentum, high pseudorapidity, and neutrinos. This data set can be further queried, and aggregations performed over it to calculate useful quantities.

For instance, the total jet transverse momentum for the W- boson may be calculated as follows:

In [26]:
anti_W_df = detect_df.query('anti_W')
pt = np.sqrt(anti_W_df['x'].sum() ** 2 + anti_W_df['y'].sum() ** 2)
pt

15.523559097491548